<a href="https://colab.research.google.com/github/Mark5484/sales-forecast-lowcode/blob/main/Low_Code_SuperKart_Model_Deployment_06172025_2312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**

## Business Context

A sales forecast is a prediction of future sales revenue based on historical data, industry trends, and the status of the current sales pipeline. Businesses use the sales forecast to estimate weekly, monthly, quarterly, and annual sales totals. A company needs to make an accurate sales forecast as it adds value across an organization and helps the different verticals to chalk out their future course of action.

Forecasting helps an organization plan its sales operations by region and provides valuable insights to the supply chain team regarding the procurement of goods and materials. An accurate sales forecast process has many benefits which include improved decision-making about the future and reduction of sales pipeline and forecast risks. Moreover, it helps to reduce the time spent in planning territory coverage and establish benchmarks that can be used to assess trends in the future.

## Objective

SuperKart is a retail chain operating supermarkets and food marts across various tier cities, offering a wide range of products. To optimize its inventory management and make informed decisions around regional sales strategies, SuperKart wants to accurately forecast the sales revenue of its outlets for the upcoming quarter.

To operationalize these insights at scale, the company has partnered with a data science firm—not just to build a predictive model based on historical sales data, but to develop and deploy a robust forecasting solution that can be integrated into SuperKart’s decision-making systems and used across its network of stores.

## Data Description

The data contains the different attributes of the various products and stores.The detailed data dictionary is given below.

- **Product_Id** - unique identifier of each product, each identifier having two letters at the beginning followed by a number.
- **Product_Weight** - weight of each product
- **Product_Sugar_Content** - sugar content of each product like low sugar, regular and no sugar
- **Product_Allocated_Area** - ratio of the allocated display area of each product to the total display area of all the products in a store
- **Product_Type** - broad category for each product like meat, snack foods, hard drinks, dairy, canned, soft drinks, health and hygiene, baking goods, bread, breakfast, frozen foods, fruits and vegetables, household, seafood, starchy foods, others
- **Product_MRP** - maximum retail price of each product
- **Store_Id** - unique identifier of each store
- **Store_Establishment_Year** - year in which the store was established
- **Store_Size** - size of the store depending on sq. feet like high, medium and low
- **Store_Location_City_Type** - type of city in which the store is located like Tier 1, Tier 2 and Tier 3. Tier 1 consists of cities where the standard of living is comparatively higher than its Tier 2 and Tier 3 counterparts.
- **Store_Type** - type of store depending on the products that are being sold there like Departmental Store, Supermarket Type 1, Supermarket Type 2 and Food Mart
- **Product_Store_Sales_Total** - total revenue generated by the sale of that particular product in that particular store


# **Please read the instructions carefully before starting the project.**

This is a commented Python Notebook file in which all the instructions and tasks to be performed are mentioned.
* Blanks '_____' are provided in the notebook that
needs to be filled with an appropriate code to get the correct result. With every '_____' blank, there is a comment that briefly describes what needs to be filled in the blank space.
* Identify the task to be performed correctly, and only then proceed to write the required code.
* Please run the codes in a sequential manner from the beginning to avoid any unnecessary errors.
* Add the results/observations (wherever mentioned) derived from the analysis in the presentation and submit the same. Any mathematical or computational details which are a graded part of the project can be included in the Appendix section of the presentation.

# **Installing and Importing the necessary libraries**

In [ ]:
#Installing the libraries with the specified versions
!pip install numpy==2.0.2 pandas==2.2.2 scikit-learn==1.6.1 matplotlib==3.10.0 seaborn==0.13.2 joblib==1.4.2 xgboost==2.1.4 requests==2.32.3 huggingface_hub==0.30.1 -q

---

### ✅ START HERE AFTER RUNTIME RESTART

Run this cell and all below (Right-click > "Run this cell and all below")

---

  

**Note:**

- After running the above cell, kindly restart the notebook kernel (for Jupyter Notebook) or runtime (for Google Colab) and run all cells sequentially from the next cell.

- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

In [ ]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd

# For splitting the dataset
from sklearn.model_selection import train_test_split

# Libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 100)


# Libraries different ensemble classifiers
from sklearn.ensemble import (
    BaggingRegressor,
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

# Libraries to get different metric scores
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)

# To create the pipeline
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline,Pipeline

# To tune different models and standardize
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# To serialize the model
import joblib

# os related functionalities
import os

# API request
import requests

# for hugging face space authentication to upload files
from huggingface_hub import login, HfApi

import warnings
warnings.filterwarnings("ignore")

# **Loading the dataset**

In [ ]:
# Uncomment the below snippet of code if the drive needs to be mounted
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
kart = pd.read_csv("https://raw.githubusercontent.com/Mark5484/sales-forecast-lowcode/refs/heads/main/SuperKart%20Data.csv")

In [ ]:
# copying data to another variable to avoid any changes to original data
data = kart.copy()

# **Data Overview**

## View the first and last 5 rows of the dataset

In [ ]:
data.head() #Complete the code to display the first 5 rows of the dataset

In [ ]:
data.tail() #Complete the code to display the last 5 rows of the dataset

## Understand the shape of the dataset

In [ ]:
print(f"There are {data.shape[0]} rows and {data.shape[1]} columns.")


## Check the data types of the columns for the dataset

In [ ]:
data.info()

## Statistical summary of the data

**Let's check the statistical summary of the data.**

In [ ]:
data.describe(include="all").T

## Checking for duplicate values

In [ ]:
# checking for duplicate values
data.duplicated().sum()

## Checking for missing values

In [ ]:
# checking for missing values in the data
data.isnull().sum()

# **Exploratory Data Analysis (EDA)**

## Univariate Analysis

In [ ]:
# function to plot a boxplot and a histogram along the same scale.

def histogram_boxplot(data, feature, figsize=(12, 7), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to the show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a star will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins, palette="winter"
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

### **Product_Weight**

In [ ]:
histogram_boxplot(data, "Product_Weight")

### **Product_Allocated_Area**

In [ ]:
histogram_boxplot(data, "Product_Allocated_Area") #Complete the code to plot the boxplot and histogram of Product_Allocated_Area

### **Product_MRP**

In [ ]:
histogram_boxplot(data, "Product_MRP") #Complete the code to plot the boxplot and histogram of Product_MRP

### **Product_Store_Sales_Total**

In [ ]:
histogram_boxplot(data, "Product_Store_Sales_Total") #Complete the code to plot the boxplot and histogram of Product_Store_Sales_Total

In [ ]:
# function to create labeled barplots


def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 1, 5))
    else:
        plt.figure(figsize=(n + 1, 5))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n].sort_values(),
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

### **Product_Sugar_Content**

In [ ]:
labeled_barplot(data, "Product_Sugar_Content", perc=True)

### **Product_Type**

In [ ]:
labeled_barplot(data, "Product_Type", perc=True) #Complete the code to plot the labelled barplot of Product_Type with the percentages being displayed

### **Store_Id**

In [ ]:
labeled_barplot(data, "Store_Id", perc=True) #Complete the code to plot the labelled barplot of Store_Id with the percentages being displayed

### **Store_Size**

In [ ]:
labeled_barplot(data, "Store_Size", perc=True) #Complete the code to plot the labelled barplot of Store_Size with the percentages being displayed

### **Store_Location_City_Type**

In [ ]:
labeled_barplot(data, "Store_Location_City_Type", perc=True) #Complete the code to plot the labelled barplot of Store_Location_City_Type with the percentages being displayed

### **Store_Type**

In [ ]:
labeled_barplot(data, "Store_Type", perc=True) #Complete the code to plot the labelled barplot of Store_Type with the percentages being displayed

## Bivariate Analysis

### **Correlation matrix**

In [ ]:
cols_list = data.select_dtypes(include=np.number).columns.tolist()

plt.figure(figsize=(10, 5))
sns.heatmap(
    data[cols_list].corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral"
)
plt.show()

### **Let's check the distribution of our target variable i.e Product_Store_Sales_Total with the numeric columns**

In [ ]:
plt.figure(figsize=[8, 6])
sns.scatterplot(x=data.Product_Weight, y=data.Product_Store_Sales_Total)
plt.show()

In [ ]:
plt.figure(figsize=[8, 6])
sns.scatterplot(x=data.Product_Allocated_Area, y=data.Product_Store_Sales_Total) #Complete the code to plot a scatterplot of Product_Allocated_Area and Product_Store_Sales_Total
plt.show()

In [ ]:
plt.figure(figsize=[8, 6])
sns.scatterplot(x=data.Product_MRP, y=data.Product_Store_Sales_Total) #Complete the code to plot a scatterplot of Product_MRP and Product_Store_Sales_Total
plt.show()

### **Let us see from which product type the company is generating most of the revenue**

In [ ]:
df_revenue1 = data.groupby(["Product_Type"], as_index=False)[
    "Product_Store_Sales_Total"
].sum()
plt.figure(figsize=[14, 8])
plt.xticks(rotation=90)
a = sns.barplot(x=df_revenue1.Product_Type, y=df_revenue1.Product_Store_Sales_Total)
a.set_xlabel("Product Types")
a.set_ylabel("Revenue")
plt.show()

In [ ]:
df_revenue2 = data.groupby(["Product_Sugar_Content"], as_index=False)[
    "Product_Store_Sales_Total"
].sum()  #Complete the code to perform a groupby on Product_Sugar_Content and select Product_Store_Sales_Total
plt.figure(figsize=[8, 6])
plt.xticks(rotation=90)
b = sns.barplot(
    x=df_revenue2.Product_Sugar_Content, y=df_revenue2.Product_Store_Sales_Total
)
b.set_xlabel("Product_Sugar_content")
b.set_ylabel("Revenue")
plt.show()

### **Let us see from which type of stores and locations the revenue generation is more**.

In [ ]:
df_store_revenue = data.groupby(["Store_Id"], as_index=False)[
    "Product_Store_Sales_Total"
].sum() #Complete the code to perform a groupby on Store_Id and select Product_Store_Sales_Total
plt.figure(figsize=[8, 6])
plt.xticks(rotation=90)
r = sns.barplot(
    x=df_store_revenue.Store_Id, y=df_store_revenue.Product_Store_Sales_Total
)
r.set_xlabel("Stores")
r.set_ylabel("Revenue")
plt.show()

In [ ]:
df_revenue3 = data.groupby(["Store_Size"], as_index=False)[
    "Product_Store_Sales_Total"
].sum() #Complete the code to perform a groupby on Store_Size and select Product_Store_Sales_Total
plt.figure(figsize=[8, 6])
plt.xticks(rotation=90)
c = sns.barplot(x=df_revenue3.Store_Size, y=df_revenue3.Product_Store_Sales_Total)
c.set_xlabel("Store_Size")
c.set_ylabel("Revenue")
plt.show()

In [ ]:
df_revenue4 = data.groupby(["Store_Location_City_Type"], as_index=False)[
    "Product_Store_Sales_Total"
].sum() #Complete the code to perform a groupby on Store_Location_City_Type and select Product_Store_Sales_Total
plt.figure(figsize=[8, 6])
plt.xticks(rotation=90)
d = sns.barplot(
    x=df_revenue4.Store_Location_City_Type, y=df_revenue4.Product_Store_Sales_Total
)
d.set_xlabel("Store_Location_City_Type")
d.set_ylabel("Revenue")
plt.show()

In [ ]:
df_revenue5 = data.groupby(["Store_Type"], as_index=False)[
    "Product_Store_Sales_Total"
].sum() #Complete the code to perform a groupby on Store_Type and select Product_Store_Sales_Total
plt.figure(figsize=[8, 6])
plt.xticks(rotation=90)
e = sns.barplot(x=df_revenue5.Store_Type, y=df_revenue5.Product_Store_Sales_Total)
e.set_xlabel("Store_Type")
e.set_ylabel("Revenue")
plt.show()

### **Let's check the distribution of our target variable i.e Product_Store_Sales_Total with the other categorical columns**

In [ ]:
plt.figure(figsize=[14, 8])
sns.boxplot(data=data, x="Store_Id", y="Product_Store_Sales_Total", hue = "Store_Id")
plt.xticks(rotation=90)
plt.title("Boxplot - Store_Id Vs Product_Store_Sales_Total")
plt.xlabel("Stores")
plt.ylabel("Product_Store_Sales_Total (of each product)")
plt.show()

In [ ]:
plt.figure(figsize=[14, 8])
sns.boxplot(data = data, x = "Store_Size", y = "Product_Store_Sales_Total", hue = "Store_Size") #Complet the code to plot the boxplot with x as Store_Size , y as Product_Store_Sales_Total and hue as Store_Size
plt.xticks(rotation=90)
plt.title("Boxplot - Store_Size Vs Product_Store_Sales_Total")
plt.xlabel("Stores")
plt.ylabel("Product_Store_Sales_Total (of each product)")
plt.show()

### **Let's now try to find out some relationship between the other columns**

In [ ]:
plt.figure(figsize=[14, 8])
sns.boxplot(data = data, x = "Product_Type", y = "Product_Weight", hue = "Product_Type") #Complete the code to plot the boxplot with x as Product_Type , y as Product_Weight and hue as Product_Type
plt.xticks(rotation=90)
plt.title("Boxplot - Product_Type Vs Product_Weight")
plt.xlabel("Types of Products")
plt.ylabel("Product_Weight")
plt.show()

### **Let's find out whether there is some relationship between the weight of the product and its sugar content**

In [ ]:
plt.figure(figsize=[14, 8])
sns.boxplot(data = data, x = "Product_Sugar_Content", y = "Product_Weight", hue = "Product_Sugar_Content") #Complete the code to plot the boxplot with x as Product_Sugar_Content , y as Product_Weight and hue as Product_Sugar_Content
plt.xticks(rotation=90)
plt.title("Boxplot - Product_Sugar_Content Vs Product_Weight")
plt.xlabel("Product_Sugar_Content")
plt.ylabel("Product_Weight")
plt.show()

### **Let's analyze the sugar content of different product types**

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(
    pd.crosstab(data["Product_Sugar_Content"], data["Product_Type"]),
    annot=True,
    fmt="g",
    cmap="viridis",
)
plt.ylabel("Product_Sugar_Content")
plt.xlabel("Product_Type")
plt.show()

### **Let's find out how many items of each product type has been sold in each of the stores**

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(
    pd.crosstab(data["Store_Id"], data["Product_Type"]), #Complete the code to perform a crosstab operation between Store_Id and Product_Type
    annot=True,
    fmt="g",
    cmap="viridis",
)
plt.ylabel("Stores")
plt.xlabel("Product_Type")
plt.show()

### **Different product types have different prices. Let's analyze the trend.**

In [ ]:
plt.figure(figsize=[14, 8])
sns.boxplot(data = data, x = "Product_Type", y = "Product_MRP", hue = "Product_Type") #Complete the code to plot a boxplot with x as Product_Type , y as Product_MRP and hue as Product_Type
plt.xticks(rotation=90)
plt.title("Boxplot - Product_Type Vs Product_MRP")
plt.xlabel("Product_Type")
plt.ylabel("Product_MRP (of each product)")
plt.show()

### **Let's find out how the Product_MRP varies with the different stores**

In [ ]:
plt.figure(figsize=[14, 8])
sns.boxplot(data = data, x = "Store_Id", y = "Product_MRP", hue = "Store_Id") #Complete the code to plot the boxplot with x as Store_Id , y as Product_MRP and hue as Store_Id
plt.xticks(rotation=90)
plt.title("Boxplot - Store_Id Vs Product_MRP")
plt.xlabel("Stores")
plt.ylabel("Product_MRP (of each product)")
plt.show()

### **Let's delve deeper and do a detailed analysis of each of the stores**.

#### OUT001

In [ ]:
data.loc[data["Store_Id"] == "OUT001"].describe(include="all").T

**Observations**
- OUT001 is a store of Supermarket Type 1 which is located in a Tier 2 city and has store size as high. It was established in 1987.
- OUT001 has sold products whose MRP range from 71 to 227.
- Snack Foods have been sold the highest number of times in OUT001.
- The revenue generated from each product at OUT001 ranges from 2300 to 5000.

In [ ]:
data.loc[data["Store_Id"] == "OUT001", "Product_Store_Sales_Total"].sum()

**OUT001 has generated total revenue of 6223113 from the sales of goods.**

In [ ]:
df_OUT001 = (
    data.loc[data["Store_Id"] == "OUT001"]
    .groupby(["Product_Type"], as_index=False)["Product_Store_Sales_Total"]
    .sum()
)
plt.figure(figsize=[14, 8])
plt.xticks(rotation=90)
plt.xlabel("Product_Type")
plt.ylabel("Product_Store_Sales_Total")
plt.title("OUT001")
sns.barplot(x=df_OUT001.Product_Type, y=df_OUT001.Product_Store_Sales_Total)
plt.show()

- OUT001 has generated the highest revenue from the sale of fruits and vegetables and snack foods. Both the categories have contributed around 800000 each.

#### OUT002

In [ ]:
data.loc[data["Store_Id"] == "OUT002"].describe(include="all").T

**Observations**
- OUT002 is a food mart which is located in a Tier 3 city and has store size as small. It was established in 1998.
- OUT002 has sold products whose MRP range from 31 to 225.
- Fruits and vegetables have been sold the highest number of times in OUT002.
- The revenue generated from each product at OUT002 ranges from 33 to 2300.

In [ ]:
data.loc[data["Store_Id"] == "OUT002", "Product_Store_Sales_Total"].sum()

**OUT002 has generated total revenue of 2030910 from the sales of goods.**

In [ ]:
df_OUT002 = (
    data.loc[data["Store_Id"] == "OUT002"]
    .groupby(["Product_Type"], as_index=False)["Product_Store_Sales_Total"]
    .sum()
)
plt.figure(figsize=[14, 8])
plt.xticks(rotation=90)
plt.xlabel("Product_Type")
plt.ylabel("Product_Store_Sales_Total")
plt.title("OUT002")
sns.barplot(x=df_OUT002.Product_Type, y=df_OUT002.Product_Store_Sales_Total)
plt.show()

- OUT002 has generated the highest revenue from the sale of fruits and vegetables (~ 300000) followed by snack foods (~ 250000).

#### OUT003

In [ ]:
data.loc[data["Store_Id"] == "OUT003"].describe(include="all").T

**Observations**
- OUT003 is a Departmental store which is located in a Tier 1 city and has store size as medium. It was established in 1999.
- OUT003 has sold products whose MRP range from 86 to 266.
- Snack Foods have been sold the highest number of times in OUT003.
- The revenue generated from each product at OUT003 ranges from 3070 to 8000.

In [ ]:
data.loc[data["Store_Id"] == "OUT003", "Product_Store_Sales_Total"].sum()

**OUT003 has generated total revenue of 6673458 from the sales of goods.**

In [ ]:
df_OUT003 = (
    data.loc[data["Store_Id"] == "OUT003"]
    .groupby(["Product_Type"], as_index=False)["Product_Store_Sales_Total"]
    .sum()
)
plt.figure(figsize=[14, 8])
plt.xticks(rotation=90)
plt.xlabel("Product_Type")
plt.ylabel("Product_Store_Sales_Total")
plt.title("OUT003")
sns.barplot(x=df_OUT003.Product_Type, y=df_OUT003.Product_Store_Sales_Total)
plt.show()

- OUT003 has generated the highest revenue from the sale of snack foods followed by fruits and vegetables, both the categories contributing around 800000 each.

#### OUT004

In [ ]:
data.loc[data["Store_Id"] == "OUT004"].describe(include="all").T

**Observations**
- OUT004 is a store of Supermarket Type2 which is located in a Tier 2 city and has store size as medium. It was established in 2009.
- OUT004 has sold products whose MRP range from 83 to 198.
- Fruits and vegetables have been sold the highest number of times in OUT004.
- The revenue generated from each product at OUT004 ranges from 1561 to 5463.

In [ ]:
data.loc[data["Store_Id"] == "OUT004", "Product_Store_Sales_Total"].sum()

**OUT004 has generated total revenue of 15427583 from the sales of goods which is highest among all the 4 stores.**

In [ ]:
df_OUT004 = (
    data.loc[data["Store_Id"] == "OUT004"]
    .groupby(["Product_Type"], as_index=False)["Product_Store_Sales_Total"]
    .sum()
)
plt.figure(figsize=[14, 8])
plt.xticks(rotation=90)
plt.xlabel("Product_Type")
plt.ylabel("Product_Store_Sales_Total")
plt.title("OUT004")
sns.barplot(x=df_OUT004.Product_Type, y=df_OUT004.Product_Store_Sales_Total)
plt.show()

- OUT004 has generated the highest revenue from the sale of fruits and vegetables (~ 2500000) followed by snack foods (~ 2000000).

**Let's find out the revenue generated by the stores from each of the product types**.

In [ ]:
df1 = data.groupby(["Product_Type", "Store_Id"], as_index=False)[
    "Product_Store_Sales_Total"
].sum()
df1

- In all the product types, the revenue generated by OUT004 has been the highest which seems quite logical since around 53% of the total products were brought from this store.
- In all the product categories, the revenue generated by OUT002 has been the lowest which seems quite obvious since it is small store in a Tier 3 city.

**Let's find out the revenue generated by the stores from products having different levels of sugar content**.

In [ ]:
df2 = data.groupby(["Product_Sugar_Content", "Store_Id"], as_index=False)[
    "Product_Store_Sales_Total"
].sum()
df2

- The trend is the same as that which was present in the revenue analysis of stores for product types.

# **Data Preprocessing**

## **Replacing the values in the Product_Sugar_Content column**

We can observe that in the Product_Sugar_Content column, there are 3 types - Low Sugar, Regular and reg.

It seems quite obvious that Regular and reg are referring to the same category. So let's replace reg with Regular.

In [ ]:
# Replacing reg with Regular
data.Product_Sugar_Content.replace(to_replace=["reg"], value=["Regular"], inplace=True)

In [ ]:
data.Product_Sugar_Content.value_counts()

## **Exploring Patterns in Product_IDs**

We can see that the Product_Id column has two characters followed by a number.

Let's delve deeper and see whether they are having any relationship with the other columns or not

In [ ]:
## extracting the first two characters from the Product_Id column and storing it in another column
data["Product_Id_char"] = data["Product_Id"].str[:2]
data.head()

In [ ]:
data["Product_Id_char"].unique()

In [ ]:
data.loc[data.Product_Id_char == "FD", "Product_Type"].unique()

In [ ]:
data.loc[data.Product_Id_char == "DR", "Product_Type"].unique() #Complete the code to select the rows where Product_Id_char is DR

In [ ]:
data.loc[data.Product_Id_char == "NC", "Product_Type"].unique() #Complete the code to select the rows where Product_Id_char is NC

## **Store's Age**

A store which has been in the business for a long duration is more trustworthy than the newly established ones.

On the other hand, older stores may sometimes lack infrastructure if proper attention is not given. So let us calculate the current age of the store and incorporate that in our model.

In [ ]:
# Outlet Age
data["Store_Age_Years"] = 2025 - data.Store_Establishment_Year

## **Grouping Product Types into Perishables and Non-Perishables.**

We have 16 different product types in our dataset.

So let us make two broad categories, perishables and non perishables, in order to reduce the number of product types

In [ ]:
perishables = [
    "Dairy",
    "Meat",
    "Fruits and Vegetables",
    "Breakfast",
    "Breads",
    "Seafood",
]

In [ ]:
def change(x):
    if x in perishables:
        return "Perishables"
    else:
        return "Non Perishables"

In [ ]:
data['Product_Type_Category'] = data['Product_Type'].apply(change)

In [ ]:
data.head()

## **Outlier Check**

In [ ]:
# outlier detection using boxplot
numeric_columns = data.select_dtypes(include=np.number).columns.tolist()
numeric_columns.remove("Store_Establishment_Year")
numeric_columns.remove("Store_Age_Years")


plt.figure(figsize=(15, 12))

for i, variable in enumerate(numeric_columns):
    plt.subplot(4, 4, i + 1)
    plt.boxplot(data[variable], whis=1.5)
    plt.tight_layout()
    plt.title(variable)

plt.show()

## **Data Preparation for Modeling**

- We aim to forecast the Product_Store_Sales_Total.

- Before building the model, we'll drop unnecessary columns and encode the categorical features.

- We'll then split the data into training and testing sets to evaluate the model's performance on unseen data.

In [ ]:
data.head()

Let's remove the columns that are not required.

In [ ]:
data = data.drop(["Product_Id","Product_Type","Store_Id","Store_Establishment_Year"], axis=1) #Complete the code to drop the columns "Product_Id","Product_Type","Store_Id","Store_Establishment_Year"

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
# Separating features and the target column
X = data.drop("Product_Store_Sales_Total", axis=1) #Complete the code to drop the target variable
y = data["Product_Store_Sales_Total"] #Complete the code to select the target variable

In [ ]:
# Splitting the data into train and test sets in 70:30 ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, shuffle=True #Complete the code to define the test_size
)

In [ ]:
X_train.shape, X_test.shape

### **Data Pre-processing Pipeline**

In [ ]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_features

In [ ]:
# Create a preprocessing pipeline for the categorical features

preprocessor = make_column_transformer(
    (Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)
)

# **Model Building**

<h2><b>Note: As per the rubric, you are required to build two ML models for this project. We have provided the code required for model building for six different ML models, each under a separate markdown section.  You may choose any two models from the ones provided below, uncomment the corresponding code for the model, and then run the code to build the model, and check its performance.</b></h2>

## Define functions for Model Evaluation

- We'll fit different models on the train data and observe their performance.
- We'll try to improve that performance by tuning some hyperparameters available for that algorithm.
- We'll use GridSearchCv for hyperparameter tuning and `r_2 score` to optimize the model.
- R-square - `Coefficient of determination` is used to evaluate the performance of a regression model. It is the amount of the variation in the output dependent attribute which is predictable from the input independent variables.
- Let's start by creating a function to get model scores, so that we don't have to use the same codes repeatedly.

In [ ]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mean_absolute_percentage_error(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

## Decision Tree Model

In [ ]:
# Uncomment the below snippet of code if decision tree regressor is to be used

dtree = DecisionTreeRegressor(random_state=1)
dtree = make_pipeline(preprocessor,dtree)
dtree.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if decision tree regressor is to be used

dtree_model_train_perf = model_performance_regression(dtree, X_train, y_train)
dtree_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if decision tree regressor is to be used

dtree_model_test_perf = model_performance_regression(dtree, X_test, y_test)
dtree_model_test_perf

## Bagging Regressor

In [ ]:
# Uncomment the below snippet of code if baggingregressor is to be used

#bagging_regressor = BaggingRegressor(random_state=1)
#bagging_regressor = make_pipeline(preprocessor,bagging_regressor)
#bagging_regressor.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if baggingregressor is to be used

#bagging_regressor_model_train_perf = model_performance_regression(bagging_regressor, X_train, y_train)
#bagging_regressor_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if baggingregressor is to be used

#bagging_regressor_model_test_perf = model_performance_regression(bagging_regressor, X_test, y_test)
#bagging_regressor_model_test_perf

## Random Forest Model

In [ ]:
# Uncomment the below snippet of code if random forest regressor is to be used

#rf_estimator = RandomForestRegressor(random_state=1)
#rf_estimator = make_pipeline(preprocessor,rf_estimator)
#rf_estimator.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if random forest regressor is to be used

#rf_estimator_model_train_perf = model_performance_regression(rf_estimator, X_train, y_train)
#rf_estimator_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if random forest regressor is to be used

#rf_estimator_model_test_perf = model_performance_regression(rf_estimator, X_test, y_test)
#rf_estimator_model_test_perf

## AdaBoost Regressor

In [ ]:
# Uncomment the below snippet of code if adaboost regressor is to be used

#ab_regressor = AdaBoostRegressor(random_state=1)
#ab_regressor = make_pipeline(preprocessor,ab_regressor)
#ab_regressor.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if adaboost regressor is to be used

#ab_regressor_model_train_perf = model_performance_regression(ab_regressor, X_train, y_train)
#ab_regressor_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if adaboost regressor is to be used

#ab_regressor_model_test_perf = model_performance_regression(ab_regressor, X_test, y_test)
#ab_regressor_model_test_perf

## Gradient Boosting Regressor

In [ ]:
# Uncomment the below snippet of code if gradientboost regressor is to be used

#gb_estimator = GradientBoostingRegressor(random_state=1)
#gb_estimator = make_pipeline(preprocessor,gb_estimator)
#gb_estimator.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if gradientboost regressor is to be used

#gb_estimator_model_train_perf = model_performance_regression(gb_estimator, X_train, y_train)
#gb_estimator_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if gradientboost regressor is to be used

#gb_estimator_model_test_perf = model_performance_regression(gb_estimator, X_test, y_test)
#gb_estimator_model_test_perf

## XGBoost Regressor

In [ ]:
# Uncomment the below snippet of code if xgboost regressor is to be used

xgb_estimator = XGBRegressor(random_state=1)
xgb_estimator = make_pipeline(preprocessor,xgb_estimator)
xgb_estimator.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if xgboost regressor is to be used

xgb_estimator_model_train_perf = model_performance_regression(xgb_estimator, X_train, y_train)
xgb_estimator_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if xgboost regressor is to be used

xgb_estimator_model_test_perf = model_performance_regression(xgb_estimator, X_test, y_test)
xgb_estimator_model_test_perf

# **Model Performance Improvement - Hyperparameter Tuning**

<h2><b>Note:</b></h2>

<h2><b>1. As per the rubric, you are required to tune two ML models for this project. We have provided the code required for model building for six different ML models, each under a separate markdown section.  You may choose the two models you built previously, uncomment the corresponding code for the model, and then run the code to tune the model, and check its performance.</b></h2>
<h2><b>2. We've provided a sample parameter grid for tuning. You may add/remove parameters or parameter values to check for during tuning as per your requirements.</b></h2>

## Hyperparameter Tuning - Decision Tree

In [ ]:
# Uncomment the below snippet of code if decision tree regressor is to be used

# # Choose the type of classifier.
dtree_tuned = DecisionTreeRegressor(random_state=1)
dtree_tuned = make_pipeline(preprocessor,dtree_tuned)

# # Grid of parameters to choose from
parameters = {
    "decisiontreeregressor__max_depth": list(np.arange(2, 6)),
    "decisiontreeregressor__min_samples_leaf": [1, 3, 5],
    "decisiontreeregressor__max_leaf_nodes": [2, 3, 5, 10, 15],
    "decisiontreeregressor__min_impurity_decrease": [0.001, 0.01, 0.1],
}

# # Run the grid search
grid_obj = GridSearchCV(dtree_tuned, parameters, scoring=r2_score, cv=3, n_jobs =-1)
grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
dtree_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
dtree_tuned.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
# Uncomment the below snippet of code if decision tree regressor is to be used

dtree_tuned_model_train_perf = model_performance_regression(dtree_tuned, X_train, y_train)
dtree_tuned_model_train_perf

### Checking model performance on test set

In [ ]:
# Uncomment the below snippet of code if decision tree regressor is to be used

dtree_tuned_model_test_perf = model_performance_regression(dtree_tuned, X_test, y_test)
dtree_tuned_model_test_perf

## Hyperparameter Tuning - Bagging Regressor

In [ ]:
#Uncomment the below snippet of code if bagging regressor is to be used

# # Choose the type of regressor.
# bagging_estimator_tuned = BaggingRegressor(random_state=1)
# bagging_estimator_tuned = make_pipeline(preprocessor,bagging_estimator_tuned)

# # Grid of parameters to choose from
# parameters = {
#     "baggingregressor__max_samples": ______, #Complete the code to define the list of values to be tuned
#     "baggingregressor__max_features": _____, #Complete the code to define the list of values to be tuned
#     "baggingregressor__n_estimators": _____, #Complete the code to define the list of values to be tuned
# }

# # Run the grid search
# grid_obj = GridSearchCV(bagging_estimator_tuned, parameters, scoring=r2_score, cv=3, n_jobs = -1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# bagging_estimator_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# bagging_estimator_tuned.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
#Uncomment the below snippet of code if bagging regressor is to be used


# bagging_estimator_tuned_model_train_perf = model_performance_regression(bagging_estimator_tuned, X_train, y_train)
# bagging_estimator_tuned_model_train_perf

### Checking model performance on test set

In [ ]:
#Uncomment the below snippet of code if bagging regressor is to be used

# bagging_estimator_tuned_model_test_perf = model_performance_regression(bagging_estimator_tuned, X_test, y_test)
# bagging_estimator_tuned_model_test_perf

## Hyperparameter Tuning - Random Forest

In [ ]:
#Uncomment the below snippet of code if random forest regressor is to be used


# # Choose the type of classifier.
# rf_tuned = RandomForestRegressor(random_state=1)
# rf_tuned = make_pipeline(preprocessor,rf_tuned)

# # Grid of parameters to choose from
# parameters = {
#     "randomforestregressor__max_depth": ______, #Complete the code to define the list of values to be tuned
#     "randomforestregressor__max_features":______, #Complete the code to define the list of values to be tuned
#     "randomforestregressor__n_estimators": ______, #Complete the code to define the list of values to be tuned
# }

# # Run the grid search
# grid_obj = GridSearchCV(rf_tuned, parameters, scoring=r2_score, cv=3, n_jobs = -1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# rf_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# rf_tuned.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
#Uncomment the below snippet of code if random forest regressor is to be used

# rf_tuned_model_train_perf = model_performance_regression(rf_tuned, X_train, y_train)
# rf_tuned_model_train_perf

### Checking model performance on test set

In [ ]:
#Uncomment the below snippet of code if random forest regressor is to be used

# rf_tuned_model_test_perf = model_performance_regression(rf_tuned, X_test, y_test)
# rf_tuned_model_test_perf

## Hyperparameter Tuning - AdaBoost Regressor

In [ ]:
#Uncomment the below snippet of code if adaboost regressor is to be used

# # Choose the type of classifier.
# ab_tuned = AdaBoostRegressor(random_state=1)
# ab_tuned = make_pipeline(preprocessor,ab_tuned)
# # Grid of parameters to choose from
# parameters = {
#     "adaboostregressor__n_estimators": _____, #Complete the code to define the list of values to be tuned
#     "adaboostregressor__learning_rate": _____, #Complete the code to define the list of values to be tuned
# }


# # Run the grid search
# grid_obj = GridSearchCV(ab_tuned, parameters, scoring=r2_score, cv=3, n_jobs = -1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# ab_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# ab_tuned.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
#Uncomment the below snippet of code if adaboost regressor is to be used

# ab_tuned_model_train_perf = model_performance_regression(ab_tuned, X_train, y_train)
# ab_tuned_model_train_perf

### Checking model performance on test set

In [ ]:
#Uncomment the below snippet of code if adaboost regressor is to be used

# ab_tuned_model_test_perf = model_performance_regression(ab_tuned, X_test, y_test)
# ab_tuned_model_train_perf

## Hyperparameter Tuning - Gradient Boosting Regressor

In [ ]:
#Uncomment the below snippet of code if gradientboost regressor is to be used


# # Choose the type of classifier.
# gb_tuned = GradientBoostingRegressor(random_state=1)
# gb_tuned = make_pipeline(preprocessor,gb_tuned)

# # Grid of parameters to choose from
# parameters = {
#     "gradientboostingregressor__n_estimators": _____, #Complete the code to define the list of values to be tuned
#     "gradientboostingregressor__subsample": _____, #Complete the code to define the list of values to be tuned
#     "gradientboostingregressor__max_features": _____, #Complete the code to define the list of values to be tuned
#     "gradientboostingregressor__max_depth": _____, #Complete the code to define the list of values to be tuned
# }


# # Run the grid search
# grid_obj = GridSearchCV(gb_tuned, parameters, scoring=r2_score, cv=3, n_jobs = -1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# gb_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# gb_tuned.fit(X_train, y_train)

### Checking model performance on training set

In [ ]:
#Uncomment the below snippet of code if gradientboost regressor is to be used

# gb_tuned_model_train_perf = model_performance_regression(gb_tuned, X_train, y_train)
# gb_tuned_model_train_perf

### Checking model performance on test set

In [ ]:
#Uncomment the below snippet of code if gradientboost regressor is to be used

# gb_tuned_model_test_perf = model_performance_regression(gb_tuned, X_test, y_test)
# gb_tuned_model_test_perf

## Hyperparameter Tuning - XGBoost Regressor

In [ ]:
# Hyperparameter tuning for XGBoost
xgb_estimator_tuned = XGBRegressor(random_state=1)
xgb_estimator_tuned = make_pipeline(preprocessor, xgb_estimator_tuned)

parameters = {
    "xgbregressor__n_estimators": [100, 200, 300],
    "xgbregressor__learning_rate": [0.01, 0.1, 0.2],
    "xgbregressor__max_depth": [3, 5, 7],
    "xgbregressor__subsample": [0.8, 1],
    "xgbregressor__colsample_bytree": [0.8, 1],
}

grid_obj = GridSearchCV(xgb_estimator_tuned, parameters, scoring='r2', cv=3, n_jobs=-1)
grid_obj = grid_obj.fit(X_train, y_train)

xgb_tuned = grid_obj.best_estimator_
xgb_tuned.fit(X_train, y_train)


### Checking model performance on training set

In [ ]:
#Uncomment the below snippet of code if xgboost regressor is to be used

xgb_tuned_model_train_perf = model_performance_regression(xgb_tuned, X_train, y_train)
xgb_tuned_model_train_perf

### Checking model performance on test set

In [ ]:
#Uncomment the below snippet of code if xgboost regressor is to be used

xgb_tuned_model_test_perf = model_performance_regression(xgb_tuned, X_test, y_test)
xgb_tuned_model_test_perf

# **Model Performance Comparison, Final Model Selection, and Serialization**

In [ ]:
 # training performance comparison

models_train_comp_df = pd.concat(
    [
        dtree_model_train_perf.T, #Complete the code to define the variable name of the dataframe which stores the train performance metrics of the first model you have choosen . Eg, rf_model_train_perf
        dtree_tuned_model_train_perf.T, #Complete the code to define the variable name of the dataframe which stores the train performance metrics of the first model (tuned) you have choosen
        xgb_estimator_model_train_perf.T, #Complete the code to define the variable name of the dataframe which stores the train performance metrics of the second model you have choosen
        xgb_tuned_model_train_perf.T, #Complete the code to define the variable name of the dataframe which stores the train performance metrics of the second model (tuned) you have choosen
    ],
    axis=1,
)

models_train_comp_df.columns = ["Decision Tree (Untuned)", "Decision Tree (Tuned)", "XGBoost (Untuned)", "XGBoost (Tuned)"] #Complete the code to define the names for the models

print("Training performance comparison:")
models_train_comp_df


In [ ]:
 # test performance comparison

models_test_comp_df = pd.concat(
    [
        dtree_model_test_perf.T, #Complete the code to define the variable name of the dataframe which stores the test performance metrics of the first model you have choosen . Eg, rf_model_test_perf
        dtree_tuned_model_test_perf.T, #Complete the code to define the variable name of the dataframe which stores the test performance metrics of the first model (tuned) you have choosen
        xgb_estimator_model_test_perf.T, #Complete the code to define the variable name of the dataframe which stores the test performance metrics of the second model you have choosen
        xgb_tuned_model_test_perf.T, #Complete the code to define the variable name of the dataframe which stores the test performance metrics of the second model (tuned) you have choosen
    ],
    axis=1,
)
0
models_test_comp_df.columns = ["Decision Tree (Untuned)", "Decision Tree (Tuned)", "XGBoost (Untuned)", "XGBoost (Tuned)"] #Complete the code to define the names for the models

print("Test performance comparison:")
models_test_comp_df

Suggestion 2: Consolidate Model Saving & Verification
This change replaces several repetitive and confusing blocks for saving and loading the model with a single, clean section

In [ ]:
# --- Model Serialization ---
# We will save our final, tuned XGBoost model. This is the "brain" of our application.

# Create the backend directory if it doesn't exist to store our deployment files.
os.makedirs("backend", exist_ok=True)

# Define the path for our single, final model file.
pipeline_path = "backend/final_model_pipeline.joblib"

# Save the tuned XGBoost pipeline (which includes the preprocessor) to a file.
joblib.dump(xgb_tuned, pipeline_path)

print(f"✅ Final model pipeline saved successfully at: {pipeline_path}")

# --- Verification (Optional but Recommended) ---
# Let's load the model back to make sure it saved correctly.
loaded_pipeline = joblib.load(pipeline_path)
print("✅ Pipeline loaded successfully for verification.")

# Predict on a sample to confirm the loaded model works.
sample_prediction = loaded_pipeline.predict(X_test.head(1))
print(f"🤖 Prediction on a test sample using loaded model: {sample_prediction[0]:.2f}")

Let's try making predictions on the test set using the deserialized model.

- Please ensure that the saved model is loaded before making predictions.

- As we can see, the model can be directly used for making predictions without any retraining.

# **Deployment - Backend**

## Flask Web Framework


In [ ]:
%%writefile backend/app.py
# Import necessary libraries
import joblib
import pandas as pd
from flask import Flask, request, jsonify

# Initialize the Flask app. '__name__' is a special Python variable that gives the script a unique name.
superkart_api = Flask(__name__)

# Load our serialized model pipeline from the file we saved earlier.
# The path is relative to the app's root directory in the Docker container.
model = joblib.load("final_model_pipeline.joblib")

# Define a route for the home page, which is useful for checking if the API is running.
@superkart_api.get('/')
def home():
    return "Welcome to the SuperKart Sales Prediction API! The endpoint is running."

# Define the main prediction endpoint.
@superkart_api.post('/v1/predict')
def predict_sales():
    # Get the JSON data sent by the frontend.
    data = request.get_json()

    # Create a pandas DataFrame from the input data, ensuring the column order matches the model's training.
    input_data = pd.DataFrame([data])

    # Use the loaded model to make a prediction.
    prediction = model.predict(input_data)[0]

    # Return the prediction as a JSON response.
    return jsonify({'Sales': prediction})

# This part is for local testing and won't be used by the Gunicorn server on Hugging Face.
if __name__ == '__main__':
    superkart_api.run(debug=True)

## Dependencies File

In [ ]:
%%writefile backend/requirements.txt
pandas==2.2.2
numpy==2.0.2
scikit-learn==1.6.1
seaborn==0.13.2
joblib==1.4.2
xgboost==2.1.4
joblib==1.4.2
Werkzeug==2.2.2
flask==2.2.2
gunicorn==20.1.0
requests==2.32.3
uvicorn[standard]
streamlit==1.43.2

## Dockerfile

In [ ]:
%%writefile backend/Dockerfile
FROM python:3.9-slim

# Set the working directory inside the container
WORKDIR /app #Complete the code to mention the command in Docker to set the working directory

# Copy all files from the current directory to the container's working directory
COPY . . #Complete the code to mention the command in Docker to copy the files from the current directory to the container's working directory

# Install dependencies from the requirements file without using cache to reduce image size
RUN pip install --no-cache-dir --upgrade -r requirements.txt #Complete the code to mention the command in Docker to install dependencies

# Define the command to start the application using Gunicorn with 4 worker processes
# - `-w 4`: Uses 4 worker processes for handling requests
# - `-b 0.0.0.0:7860`: Binds the server to port 7860 on all network interfaces
# - `app:app`: Runs the Flask app (assuming `app.py` contains the Flask instance named `app`)
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:7860", "app:superkart_api"]

## Setting up a Hugging Face Docker Space for the Backend

**Note**: We are creating a Hugging Face Docker Space for our backend using the Hugging Face Hub API. This automates the space creation process and enables seamless deployment of our Flask app.

In [ ]:
# Import the login function from the huggingface_hub library
from huggingface_hub import login

# Login to your Hugging Face account using your access token
# Replace "YOUR_HUGGINGFACE_TOKEN" with your actual token
#login(token='HF_TOKEN') #Complete the code to define the access token

# Import the create_repo function from the huggingface_hub library
from huggingface_hub import create_repo

In [ ]:
from huggingface_hub import create_repo

# Try to create the repository for the Hugging Face Space
try:
    create_repo(
        "superkart-sales-backend",  # 🚨 Choose your own unique repo name!
        repo_type="space",          # Type must be "space"
        space_sdk="docker",         # Use Docker for backend
        private=False               # Set to True if you want to keep it private
    )
    print("Repository created successfully.")
except Exception as e:
    if "RepositoryAlreadyExistsError" in str(e):
        print("Repository already exists. Skipping creation.")
    else:
        print(f"Error creating repository: {e}")


## Uploading Files to Hugging Face Space (Docker Space)

**Note**: Before running the code below, ensure that the serialized ML model has been uploaded in to `backend_files` folder.

# **Deployment - Frontend**

## Streamlit for Interactive UI

In [ ]:
# Create a folder for storing the files needed for frontend UI deployment
os.makedirs("frontend", exist_ok=True) #Complete the code to define the folder name where the files related to the frontend system will be stored


In [ ]:
%%writefile frontend/app.py
import streamlit as st
import requests

st.set_page_config(layout="wide")
st.title("🛒 SuperKart Sales Predictor")

# --- IMPORTANT ---
# Replace 'YOUR_USERNAME' with your actual Hugging Face username
API_URL = "https://Mark5484-superkart-sales-backend.hf.space/v1/predict"

st.info(f"This app sends requests to a backend API running at: `{API_URL}`")

# Two columns for cleaner layout
col1, col2 = st.columns(2)

with col1:
    st.subheader("📦 Product Details")
    Product_Weight = st.number_input("Product Weight (in g)", min_value=0.0, value=12.5, format="%.2f")
    Product_Sugar_Content = st.selectbox("Product Sugar Content", ["Low Sugar", "Regular", "No Sugar"])
    Product_Allocated_Area = st.slider("Product Allocated Area", 0.01, 0.2, 0.05, help="Ratio of shelf space allocated")
    Product_MRP = st.number_input("Product MRP (₹)", min_value=0.0, value=150.0, format="%.2f")
    Product_Id_char = st.selectbox("Product ID Prefix", ["FD", "DR", "NC"])
    Product_Type_Category = st.selectbox("Product Type Category", ["Perishables", "Non Perishables"])

with col2:
    st.subheader("🏬 Store Details")
    Store_Age_Years = st.number_input("Store Age (Years)", min_value=1, value=15)
    Store_Size = st.selectbox("Store Size", ["Small", "Medium", "High"])
    Store_Location_City_Type = st.selectbox("Store Location City Type", ["Tier 1", "Tier 2", "Tier 3"])
    Store_Type = st.selectbox("Store Type", ["Supermarket Type1", "Supermarket Type2", "Supermarket Type3", "Grocery Store"])

if st.button("Predict Sales", type='primary'):
    product_data = {
        "Product_Weight": Product_Weight,
        "Product_Sugar_Content": Product_Sugar_Content,
        "Product_Allocated_Area": Product_Allocated_Area,
        "Product_MRP": Product_MRP,
        "Store_Size": Store_Size,
        "Store_Location_City_Type": Store_Location_City_Type,
        "Store_Type": Store_Type,
        "Product_Id_char": Product_Id_char,
        "Store_Age_Years": Store_Age_Years,
        "Product_Type_Category": Product_Type_Category
    }

    with st.spinner('📡 Sending request to the backend API...'):
        try:
            response = requests.post(API_URL, json=product_data)
            response.raise_for_status()
            result = response.json()
            predicted_sales = result["Sales"]
            st.success(f"💰 Predicted Product Store Sales Total: ₹{predicted_sales:,.2f}")
        except requests.exceptions.RequestException as e:
            st.error(f"❌ Error: Could not connect to the API. Details: {e}")


## Dependencies File

In [ ]:
%%writefile frontend/requirements.txt
streamlit
pandas
requests

## Setting up a HuggingFace Streamlit Space for the Frontend

**Note**: We are creating a Hugging Face Streamlit Space for our frontend, similar to the backend Flask app, using the Hugging Face Hub API. This approach ensures automation in the space creation process and enables seamless deployment of our Streamlit application.

In [ ]:
import os
import joblib
from huggingface_hub import HfApi, login
from getpass import getpass


In [ ]:
# --- Step 1: Login to Hugging Face ---
# You will need an access token with WRITE permissions from your Hugging Face account settings.

HF_TOKEN = getpass("Enter your Hugging Face token: ")
login(token=HF_TOKEN)

api = HfApi()

# --- Step 2: Define your repository IDs ---
# Replace 'YOUR_USERNAME' with your actual Hugging Face username.
BACKEND_REPO_ID = "Mark5484/superkart-sales-backend"  # Your DOCKER space
FRONTEND_REPO_ID = "Mark5484/superkart-sales-frontend" # Your STREAMLIT space

print(f"Backend will be uploaded to: {BACKEND_REPO_ID}")
print(f"Frontend will be uploaded to: {FRONTEND_REPO_ID}")

In [ ]:
# --- Step 3: Upload the Backend (Docker) ---
print("\nUploading backend files...")
api.upload_folder(
    folder_path="backend",
    repo_id=BACKEND_REPO_ID,
    repo_type="space",
    commit_message="Pushing backend Flask app files"
)
print("✅ Backend upload complete! Your API should be building now.")

# --- Step 4: Upload the Frontend (Streamlit) ---
print("\nUploading frontend files...")
api.upload_folder(
    folder_path="frontend",
    repo_id=FRONTEND_REPO_ID,
    repo_type="space",
    commit_message="Pushing frontend Streamlit app files"
)
print("✅ Frontend upload complete! Your Streamlit app should be building now.")

print(f"\n🚀 Access your frontend here: https://huggingface.co/spaces/{FRONTEND_REPO_ID}")

# **Actionable Insights and Business Recommendations**

-

-